Read in pre-computed files of RT and scheduled data and compare!

In [ ]:
# required for pandas to read csv from aws
import s3fs
import os
import pandas as pd
import pendulum
import shapely
import geopandas

In [ ]:
start_date = '2022-05-20'
end_date = '2022-07-16'

date_range = [d for d in pendulum.period(pendulum.from_format(start_date, 'YYYY-MM-DD'), pendulum.from_format(end_date, 'YYYY-MM-DD')).range('days')]

In [ ]:
schedule_raw = pd.read_csv('s3://chn-ghost-buses-private/SCHEDULED_route_daily_hourly_summary.csv')

In [ ]:
rt_raw = pd.DataFrame()

for day in date_range:
    date_str = day.to_date_string()
    print(f"Processing {date_str} at {pendulum.now().to_datetime_string()}")    
    daily_data = pd.read_csv(f's3://chn-ghost-buses-private/bus_hourly_summary_v2/{date_str}.csv')
    rt_raw = rt_raw.append(daily_data)

In [ ]:
# basic reformatting
rt = rt_raw.copy()
schedule = schedule_raw.copy()
rt['date'] = pd.to_datetime(rt.data_date, format = '%Y-%m-%d')
rt['route_id'] = rt['rt']
schedule['date'] = pd.to_datetime(schedule.date, format = '%Y-%m-%d')

In [ ]:
# get total by route by day
rt_daily_by_rte = rt.groupby(by = ['date', 'route_id'])['trip_count'].sum().reset_index()
sched_daily_by_rte = schedule.groupby(by = ['date', 'route_id'])['trip_count'].sum().reset_index()


In [ ]:
compare_daily_by_rte = rt_daily_by_rte.merge(sched_daily_by_rte, how = 'inner', 
                                             on = ['date', 'route_id'], suffixes = ['_rt', '_sched'])

In [ ]:
# compare by day of week 
compare_daily_by_rte['dayofweek'] = compare_daily_by_rte['date'].dt.dayofweek
compare_daily_by_rte['day_type'] = compare_daily_by_rte['dayofweek'].map({0: 'wk', 
                                                                         1: 'wk',
                                                                         2: 'wk',
                                                                         3: 'wk',
                                                                         4: 'wk',
                                                                         5: 'sat',
                                                                         6: 'sun'})

compare_daily_by_rte.loc[compare_daily_by_rte.date.isin(['2022-05-31', '2022-07-04']), 'day_type'] = 'hol'

In [ ]:
compare_by_day_type = compare_daily_by_rte.groupby(['route_id', 'day_type'])[['trip_count_rt', 'trip_count_sched']].sum().reset_index()

In [ ]:
compare_by_day_type['ratio'] = compare_by_day_type['trip_count_rt'] / compare_by_day_type['trip_count_sched']

In [ ]:
compare_by_day_type.sort_values(by = ['day_type', 'route_id'])

In [ ]:
compare_by_day_type[compare_by_day_type.day_type == 'wk'].sort_values(by = 'ratio')

In [ ]:
compare_by_day_type.to_csv('initial_comparison.csv')